Using Content Based Filtering to recommend similar items by:

* Creating user-feature matrix
* Creating item-feature matrix
* Measuring similraity using dot product as metric
* Recommending top-k similar items

Second Approach

* Perform dimensionality reduction using PCA on user_feature and item_feature matrices

I used rows from transactions record

Limited customers to customers who bought at least two items

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go

In [3]:
sale_path = 'sales_and_inventory_mentor_data\Sales_mentor\TT T03-2022_split_2.xlsx'
products_path = 'sales_and_inventory_mentor_data\MasterData\Productmaster.xlsx'

In [6]:
sale_df = pd.read_excel(sale_path)
sale_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87684 entries, 0 to 87683
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   month                      87684 non-null  int64 
 1   week                       87684 non-null  int64 
 2   site                       87684 non-null  int64 
 3   branch_id                  87684 non-null  int64 
 4   channel_id                 87684 non-null  object
 5   distribution_channel       87684 non-null  object
 6   distribution_channel_code  87684 non-null  object
 7   sold_quantity              87684 non-null  int64 
 8   cost_price                 87684 non-null  int64 
 9   net_price                  87684 non-null  int64 
 10  customer_id                87684 non-null  object
 11  product_id                 87684 non-null  object
dtypes: int64(7), object(5)
memory usage: 8.0+ MB


In [7]:
product_df = pd.read_excel(products_path)
product_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94867 entries, 0 to 94866
Data columns (total 32 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Unnamed: 0            94867 non-null  int64  
 1   index                 94867 non-null  int64  
 2   color                 94822 non-null  object 
 3   color_group           94867 non-null  object 
 4   listing_price         94840 non-null  float64
 5   price_group           94840 non-null  object 
 6   gender                94867 non-null  object 
 7   product_group         94867 non-null  object 
 8   detail_product_group  94867 non-null  object 
 9   shoe_product          94867 non-null  object 
 10  size_group            94867 non-null  object 
 11  size                  94722 non-null  float64
 12  age_group             94867 non-null  object 
 13  activity_group        94867 non-null  object 
 14  collection            94862 non-null  object 
 15  image_copyright    

In [8]:
users = sale_df
df = users.merge(product_df, on='product_id')
df.head()

,month,week,site,branch_id,channel_id,distribution_channel,distribution_channel_code,sold_quantity,cost_price_x,net_price,...,mc_description,name_description,mch_description,code_lock,option,cost_price_y,product_syle_color,product_syle,brand_name,vendor_name
0,2022003,202211,1200,1200,TGPP,Phát sinh,ZF2,1,203629,299455,...,Hàng chậm,"SAN DAL TPR NỮ 88, Đen, 37",NHÓM HÀNG CHẬM,NaN,P02-C02,207400.0,4576f65145124b6fb2884adfa197003dDEN,4576f65145124b6fb2884adfa197003d,Brand1,vendor0
1,2022003,202212,1151,1100,CHTT,Bán lẻ,FP,1,203629,299000,...,Hàng chậm,"SAN DAL TPR NỮ 88, Đen, 37",NHÓM HÀNG CHẬM,NaN,P02-C02,207400.0,4576f65145124b6fb2884adfa197003dDEN,4576f65145124b6fb2884adfa197003d,Brand1,vendor0
2,2022003,202212,1150,1100,CHTT,Bán lẻ,FP,1,120174,177000,...,Dép Xốp Nữ,"Dép Xốp Nữ, Nâu, 37",NHÓM XỐP,NaN,P02-C05,129200.0,a42395a564364e5693b178dce514bd84NAU,a42395a564364e5693b178dce514bd84,Brand1,vendor0
3,2022003,202210,1513,1500,CHTT,Bán lẻ,FP,1,120174,177000,...,Dép Xốp Nữ,"Dép Xốp Nữ, Nâu, 37",NHÓM XỐP,NaN,P02-C05,129200.0,a42395a564364e5693b178dce514bd84NAU,a42395a564364e5693b178dce514bd84,Brand1,vendor0
4,2022003,202213,1101,1100,CHTT,Bán sỉ,FP,1,120174,146910,...,Dép Xốp Nữ,"Dép Xốp Nữ, Nâu, 37",NHÓM XỐP,NaN,P02-C05,129200.0,a42395a564364e5693b178dce514bd84NAU,a42395a564364e5693b178dce514bd84,Brand1,vendor0


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87684 entries, 0 to 87683
Data columns (total 43 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   month                      87684 non-null  int64  
 1   week                       87684 non-null  int64  
 2   site                       87684 non-null  int64  
 3   branch_id                  87684 non-null  int64  
 4   channel_id                 87684 non-null  object 
 5   distribution_channel       87684 non-null  object 
 6   distribution_channel_code  87684 non-null  object 
 7   sold_quantity              87684 non-null  int64  
 8   cost_price_x               87684 non-null  int64  
 9   net_price                  87684 non-null  int64  
 10  customer_id                87684 non-null  object 
 11  product_id                 87684 non-null  object 
 12  Unnamed: 0                 87684 non-null  int64  
 13  index                      87684 non-null  int

In [10]:
feature_subset = ['color_group', 'price_group', 'detail_product_group', 'size_group', 'age_group', 'activity_group', 'lifestyle_group', 'brand_name']

In [11]:
df1 = df[['customer_id', 'product_id'] + feature_subset]
dummies_df = pd.get_dummies(df1, columns=feature_subset)
dummies_df

,customer_id,product_id,color_group_KHÁC,color_group_NHIỀU MÀU,color_group_SÁNG,color_group_TRẮNG,color_group_TỐI,color_group_ĐEN,color_group_ĐEN,price_group_100<200,...,activity_group_Văn phòng,lifestyle_group_Casual,lifestyle_group_Fashion,lifestyle_group_Formal,lifestyle_group_Khác,lifestyle_group_Sport,brand_name_Brand1,brand_name_Brand2,brand_name_Brand3,brand_name_Brand4
0,c1cfc02d2,4576f65145124b6fb2884adfa197003dDEN37,False,False,False,False,False,True,False,True,...,False,True,False,False,False,False,True,False,False,False
1,809d3959a,4576f65145124b6fb2884adfa197003dDEN37,False,False,False,False,False,True,False,True,...,False,True,False,False,False,False,True,False,False,False
2,3389917a2,a42395a564364e5693b178dce514bd84NAU37,False,False,False,False,True,False,False,True,...,False,True,False,False,False,False,True,False,False,False
3,3ec77a1f4,a42395a564364e5693b178dce514bd84NAU37,False,False,False,False,True,False,False,True,...,False,True,False,False,False,False,True,False,False,False
4,f56cfa95a,a42395a564364e5693b178dce514bd84NAU37,False,False,False,False,True,False,False,True,...,False,True,False,False,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87679,367a5f155,87bbc837d0d0437bb157745b5d3c5c0bDOO30,False,False,True,False,False,False,False,False,...,False,True,False,False,False,False,True,False,False,False
87680,6e1a4fb79,98588d09c1bc4646a8c35d7244160831DOO35,False,False,False,False,True,False,False,False,...,False,True,False,False,False,False,True,False,False,False
87681,a0c1d36b9,a6339b0b19764242ad153c0e1c3596b6DEN35,False,False,False,False,False,True,False,True,...,False,False,True,False,False,False,True,False,False,False
87682,167a78f6f,50fd98b1465343948af6ab21ae090d02DEN38,False,False,False,False,False,True,False,False,...,True,False,False,True,False,False,True,False,False,False


In [12]:
minimum_items = 1
groupby_customer = dummies_df.groupby('customer_id')

l = []
cutomer_ids = []
article_ids = []
for key in groupby_customer.groups.keys():
    temp = groupby_customer.get_group(key)
    if temp.product_id.nunique() >= minimum_items:
        l.append(temp.drop('product_id', axis=1).sum(numeric_only=True).values)
        cutomer_ids.append(key)
        article_ids.extend(temp.product_id.values.tolist())

In [13]:
user_feature = pd.DataFrame(l, columns = dummies_df.columns[2:])
normalized_user_feature = user_feature.div(user_feature.sum(axis=1), axis=0)
normalized_user_feature.insert(0, 'customer_id', cutomer_ids)
normalized_user_feature = normalized_user_feature.set_index('customer_id')
normalized_user_feature

,color_group_KHÁC,color_group_NHIỀU MÀU,color_group_SÁNG,color_group_TRẮNG,color_group_TỐI,color_group_ĐEN,color_group_ĐEN,price_group_100<200,price_group_200<300,price_group_300<600,...,activity_group_Văn phòng,lifestyle_group_Casual,lifestyle_group_Fashion,lifestyle_group_Formal,lifestyle_group_Khác,lifestyle_group_Sport,brand_name_Brand1,brand_name_Brand2,brand_name_Brand3,brand_name_Brand4
customer_id,,,,,,,,,,,,,,,,,,,,,
004c2ae83,0.003531,0.032486,0.030367,0.000000,0.055085,0.003531,0.000000,0.040254,0.031780,0.033898,...,0.000000,0.092514,0.000000,0.000000,0.000000,0.032486,0.117232,0.007768,0.0,0.0
007e6bb72,0.000000,0.000000,0.038690,0.000000,0.068452,0.014881,0.002976,0.038690,0.041667,0.044643,...,0.000000,0.104167,0.000000,0.000000,0.000000,0.020833,0.125000,0.000000,0.0,0.0
008e9d4fd,0.000000,0.008212,0.020985,0.005474,0.072993,0.017336,0.000000,0.033759,0.016423,0.047445,...,0.000000,0.057482,0.000000,0.000000,0.000000,0.067518,0.102190,0.022810,0.0,0.0
00981a197,0.000000,0.000000,0.000000,0.000000,0.125000,0.000000,0.000000,0.000000,0.070312,0.054688,...,0.000000,0.023438,0.000000,0.000000,0.000000,0.101562,0.125000,0.000000,0.0,0.0
01347a866,0.000000,0.010870,0.000000,0.000000,0.092391,0.021739,0.000000,0.054348,0.010870,0.005435,...,0.000000,0.114130,0.000000,0.000000,0.000000,0.010870,0.114130,0.010870,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
fc0dc8117,0.002016,0.021169,0.031250,0.003024,0.054435,0.013105,0.000000,0.027218,0.033266,0.035282,...,0.004032,0.081653,0.001008,0.003024,0.004032,0.035282,0.111895,0.013105,0.0,0.0
fc1944a2b,0.000000,0.000000,0.015957,0.000000,0.077128,0.021277,0.010638,0.023936,0.050532,0.039894,...,0.000000,0.090426,0.013298,0.000000,0.000000,0.021277,0.122340,0.002660,0.0,0.0
fc5887d8c,0.000000,0.000000,0.000000,0.000000,0.093750,0.031250,0.000000,0.125000,0.000000,0.000000,...,0.000000,0.125000,0.000000,0.000000,0.000000,0.000000,0.125000,0.000000,0.0,0.0


In [14]:
item_feature = dummies_df.drop_duplicates(subset='product_id')
item_feature = item_feature[item_feature.product_id.isin(article_ids)].drop('customer_id', axis=1)
item_feature = item_feature.set_index('product_id')
item_feature

,color_group_KHÁC,color_group_NHIỀU MÀU,color_group_SÁNG,color_group_TRẮNG,color_group_TỐI,color_group_ĐEN,color_group_ĐEN,price_group_100<200,price_group_200<300,price_group_300<600,...,activity_group_Văn phòng,lifestyle_group_Casual,lifestyle_group_Fashion,lifestyle_group_Formal,lifestyle_group_Khác,lifestyle_group_Sport,brand_name_Brand1,brand_name_Brand2,brand_name_Brand3,brand_name_Brand4
product_id,,,,,,,,,,,,,,,,,,,,,
4576f65145124b6fb2884adfa197003dDEN37,False,False,False,False,False,True,False,True,False,False,...,False,True,False,False,False,False,True,False,False,False
a42395a564364e5693b178dce514bd84NAU37,False,False,False,False,True,False,False,True,False,False,...,False,True,False,False,False,False,True,False,False,False
07352d9ab2e4403b823219f660d2d1baXAM42,False,False,False,False,True,False,False,True,False,False,...,False,False,False,False,False,True,True,False,False,False
43fcaa7920db44cd8f044fb58d1d27e0XAM42,False,False,False,False,True,False,False,True,False,False,...,False,True,False,False,False,False,True,False,False,False
d4281d34e5674d74ae92ff4c38dfc27eXMN40,False,False,False,False,True,False,False,False,False,True,...,False,False,False,False,False,True,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87bbc837d0d0437bb157745b5d3c5c0bDOO30,False,False,True,False,False,False,False,False,False,False,...,False,True,False,False,False,False,True,False,False,False
98588d09c1bc4646a8c35d7244160831DOO35,False,False,False,False,True,False,False,False,False,False,...,False,True,False,False,False,False,True,False,False,False
a6339b0b19764242ad153c0e1c3596b6DEN35,False,False,False,False,False,True,False,True,False,False,...,False,False,True,False,False,False,True,False,False,False


In [15]:
scores = normalized_user_feature.dot(item_feature.T)
scores

product_id,4576f65145124b6fb2884adfa197003dDEN37,a42395a564364e5693b178dce514bd84NAU37,07352d9ab2e4403b823219f660d2d1baXAM42,43fcaa7920db44cd8f044fb58d1d27e0XAM42,d4281d34e5674d74ae92ff4c38dfc27eXMN40,2785d5fa66d943e9be5c16d67971f86dKEM36,e4ca5abb177d49d9aae8907448b24ed1CAM42,9b09272773324deb8e1d1a131659b39fXAM43,162324471a4a4e6ca051b6190f03f218DEN38,35c91faa5b8040cbbb88e101f0739ba3DEN35,...,9ce96ee3a68e480686d7e44e1449ee72DEN40,9d807cda3ef54886b273c9cface6a1e7XNG37,a3565039239a42bb95b602e26e81103aNAD30,e41a36e824fd40d2a0b7a36d39049484HOG38,8ea2029e1b4d4b1cb501b177cca36449TRG42,87bbc837d0d0437bb157745b5d3c5c0bDOO30,98588d09c1bc4646a8c35d7244160831DOO35,a6339b0b19764242ad153c0e1c3596b6DEN35,50fd98b1465343948af6ab21ae090d02DEN38,22826feb50d84a2eb8c71cf03c7e8f05DEN38
customer_id,,,,,,,,,,,,,,,,,,,,,
004c2ae83,0.511299,0.522599,0.418785,0.562853,0.543785,0.44774,0.532486,0.28178,0.532486,0.509887,...,0.464689,0.622881,0.293079,0.249294,0.536723,0.573446,0.509887,0.334746,0.20339,0.415254
007e6bb72,0.53869,0.547619,0.4375,0.592262,0.568452,0.497024,0.571429,0.270833,0.571429,0.532738,...,0.511905,0.577381,0.28869,0.258929,0.571429,0.5625,0.532738,0.363095,0.244048,0.380952
008e9d4fd,0.437044,0.515511,0.581204,0.492701,0.488139,0.480839,0.573905,0.440693,0.573905,0.548358,...,0.47354,0.386861,0.287409,0.256387,0.520073,0.42792,0.548358,0.406022,0.30292,0.476277
00981a197,0.398438,0.453125,0.640625,0.523438,0.671875,0.4375,0.507812,0.460938,0.507812,0.507812,...,0.382812,0.484375,0.375,0.179688,0.523438,0.46875,0.507812,0.359375,0.289062,0.53125
01347a866,0.576087,0.73913,0.608696,0.646739,0.369565,0.565217,0.820652,0.391304,0.820652,0.815217,...,0.646739,0.418478,0.331522,0.309783,0.711957,0.532609,0.815217,0.521739,0.347826,0.451087
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
fc0dc8117,0.498992,0.517137,0.478831,0.540323,0.467742,0.509073,0.565524,0.329637,0.565524,0.552419,...,0.488911,0.479839,0.303427,0.273185,0.543347,0.524194,0.552419,0.394153,0.301411,0.416331
fc1944a2b,0.505319,0.603723,0.553191,0.56117,0.446809,0.577128,0.680851,0.335106,0.680851,0.667553,...,0.574468,0.43883,0.337766,0.271277,0.632979,0.515957,0.667553,0.476064,0.367021,0.43883
fc5887d8c,0.65625,0.84375,0.71875,0.71875,0.34375,0.625,0.96875,0.34375,0.96875,0.84375,...,0.65625,0.375,0.34375,0.375,0.71875,0.5,0.84375,0.65625,0.40625,0.40625


In [16]:
def get_rcmnd(customer_id, scores):
    cutomer_scores = scores.loc[customer_id]
    customer_prev_items = groupby_customer.get_group(customer_id)['product_id']
    prev_dropped = cutomer_scores.drop(customer_prev_items.values)
    ordered = prev_dropped.sort_values(ascending=False)
    return ordered, customer_prev_items

In [17]:
k = 6
customer_id = scores.index[1]
rcmnds, prev_items = get_rcmnd(customer_id, scores)

In [18]:
rcmnds[:k]

product_id
a9aca9eb453047d1bfc222e67d1b44d3DEN33    0.690476
b7ca367b65204b6489d8b75b9279a98aDOO34    0.690476
a4c9e5a9a66841fbbed65489963128f1DEN35    0.690476
7f7bfd510f4f4eca8c0dd976edd22e58DOO37    0.690476
56f0f79767744832b6ed9f8bfc0dd3deXAM35    0.690476
124c81c04c264be19d5f689e34e7651dDEN30    0.690476
Name: 007e6bb72, dtype: object

In [19]:
prev_items

1929     8233e0b7686c4d5495bbedf3dd61ae27NAD43
6602     d66c00144c944ecc9138f027a49aaac4XDG34
8804     0f605b5e166c45c2bf802b83fd45e17bXDD35
10084    217e14c3caa64cc98a74613925282989NAU42
10391    00d566412e8e490e80b390f41d133496KEM36
12157    61e0957407c1462e937bcec012e516f9DEN30
13833    5e7f23bd00f342cdad9c3371d1faf2f7KEM39
16539    cad7c31274914646813fce5e7cbda200DEN40
23187    a7ba172c348d47af88c9439e9ac9ee6bHOG33
25701    000108ac94864037b89c6d7ce78d1134KEM35
26727    cf332e911e634680b68140625339cfa5DEN43
26832    c62e21fa856b4a47a01d9252f547bb4bDEN38
28978    cef019e4eabc485baaacd9fee899ac40XAM28
30247    4c32d7c07b9b42aeb8c6e57357a497f6NAU38
31725    9a1e9a472c214a71ad4be45c3130acf3DEN31
34622    e4bfb130d7fc4be0b7277d3dfa25a25eXDD31
40951    91f827730787410b9212b9827d4f4c96XNH31
41423    51262e710bbc4ad3b254853a7560c447DEN39
42003    a5ed4e172f414260803f18dc8e8ef8d9NAD40
43134    272632b4f8d948a0b58d8dd787497772DOO31
44864    4277369e4fa54b35ab23e1959af9cc6aDOO28
48444    48eb